#  Keras - Python Deep Learning Neural Network API

https://deeplizard.com/learn/playlist/PLZbbT5o_s2xrwRnXk_yCPtnqqo4_u2YGL

### Now Keras became integrated with the TensorFlow library
#### Import statement have been changed

In [ ]:
#Before

import keras
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D

In [ ]:
#After

# -- For Build a Neural Network --
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Conv2D

# -- For Training a Neural Network --  
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Expected Data Format

The Sequential model receives data during training, which occurs when we call the fit() function on the model. Therefore, we need to check the type of data this function expects.

https://www.tensorflow.org/api_docs/python/tf/keras/Sequential#fit

 the input data x in fit() need to be one of the following data types.

- A Numpy array (or array-like), or a list of arrays (in case the model has multiple inputs).
- A TensorFlow tensor, or a list of tensors (in case the model has multiple inputs).
- A dict mapping input names to the corresponding array/tensors, if the model has named inputs.

### Test/Train Split 

##### from udemy course

In [ ]:
from sklearn.model_selection import train_test_split

# Features
X = df[['feature1','feature2']].values

# Label
y = df['price'].values

# Split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

### Data Processing (Normalizing / Scaling Data) 

use scikit-learn’s **MinMaxScaler** class to scale all of the data down from a scale ranging from 13 to 100 to be on a scale from 0 to 1.

standardization and normalization techniques https://deeplizard.com/learn/video/dXB-KQYkzNU

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
scaled_train_samples = scaler.fit_transform(train_samples.reshape(-1,1))

We reshape the data as a technical requirement just since the fit_transform() function doesn’t accept 1D data by default.

##### from Udemy course 

In [ ]:
scaler = MinMaxScaler()
X_train= scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

****

## Create An Artificial Neural Network 

create a simple artificial neural network using a Sequential model from the Keras API 

### Build A Sequential Model 

#### 1. As a list of Layer 

In [ ]:
model = Sequential([
    Dense(units=16, input_shape=(1,), activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=2, activation='softmax')
])

Our first layer is a Dense layer, next layer will also be a Dense layer and Lastly we specify the output layer


The first required parameter that the Dense layer expects is the number of neurons or units the layer has, and we’re arbitrarily setting this to 16

The parameter called **input_shape** is how we specify the shape of the input data (Specify to the first hidden layer)

(1,) as the input_shape of our one-dimensional data

***

#### 2. Adding layers one by one 

In [ ]:
model = Sequential()

model.add(Dense(4,activation='relu'))
model.add(Dense(4,activation='relu'))
model.add(Dense(4,activation='relu'))

# Final output node for prediction
model.add(Dense(1))


We can call **summary()** on our model to get a quick visualization of it.

In [ ]:
model.summary()

we need to do to get the model ready for training is call the **compile()** function on it.

"Compile" =  configures the Sequential model

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy']
             )

Note that we could instead configure our output layer to have only one output, rather than two, and use **binary_crossentropy as our loss** however, the **last layer would need to use sigmoid**, rather than softmax

#### Choosing Optimizer and Lost 

Keep in mind what kind of problem you are trying to solve:

    # For a multi-class classification problem
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # For a binary classification problem
    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # For a mean squared error regression problem
    model.compile(optimizer='rmsprop',
                  loss='mse')
                  
***

## Training The Model 
#### we can train it using the fit() function.

In [ ]:
model.fit(x=scaled_train_samples, y=train_labels, batch_size=10, epochs=30, verbose=2)

Verbose  specifies how much output to the console we want to see during each epoch of training. The verbosity levels range from 0 to 2

## Build A Validation Set
### Using Validation_set

Before training begins, we can choose to remove a portion of the training set and place it in a validation set

This also **helps us see whether or not the model is overfitting** **Overfitting occurs when the model only learns the specifics of the training data** and is unable to generalize well on data that it wasn’t trained on.

#### 1. Manually Create Validation Set 

In [ ]:
model.fit(
      x=scaled_train_samples
    , y=train_labels
    , validation_data=valid_set
    , batch_size=10
    , epochs=30
    , verbose=2
)

This data structure should be a tuple **valid_set = (x_val, y_val)** of Numpy arrays or tensors, where 

- x_val or x_test is a numpy array or tensor containing validation samples, and 
- y_val or y_test is a numpy array or tensor containing validation labels.

#### 2. Create Validation Set With Keras 

In [ ]:
model.fit(
      x=scaled_train_samples
    , y=train_labels
    , validation_split=0.1
    , batch_size=10
    , epochs=30
    , verbose=2
)

If we don’t already have a specified validation set created, we can set a value for the **validation_split parameter**. It expects a fractional number between 0 and 1. Suppose that we set this parameter to 0.1.

### Training too much epochs 
#### Leads to overfitting

2 ways to deal with the problem
- Early Stopping
- adding Dropout layers

In [ ]:
model.fit(x=X_train, 
          y=y_train, 
          epochs=600,
          validation_data=(X_test, y_test), verbose=1
          )

model_loss = pd.DataFrame(model.history.history)
model_loss.plot()

<img src='img\overfit.png'/>

### Early stopping
#### stop when "convergence", which means when loss is not apparently changed then stop the leaning iteration

In [ ]:
model = Sequential()
model.add(Dense(units=30,activation='relu'))
model.add(Dense(units=15,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')


from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)

model.fit(x=X_train, 
          y=y_train, 
          epochs=600,
          validation_data=(X_test, y_test), verbose=1,
          callbacks=[early_stop]
          )

In [ ]:
model_loss = pd.DataFrame(model.history.history)
model_loss.plot()

<img src='img\early_stop.png'/>

### Adding Dropout layer
#### เราสามารถใช้แค่โมเดลเดียว มาจำลองเป็นหลาย ๆ โมเดลได้ โดยการสุ่มถอดบาง Node ออก ในระหว่างการเทรน วิธีนี้เรียกว่า Dropout

In [ ]:
from tensorflow.keras.layers import Dropout

model = Sequential()
model.add(Dense(units=30,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=15,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')



model.fit(x=X_train, 
          y=y_train, 
          epochs=600,
          validation_data=(X_test, y_test), verbose=1,
          callbacks=[early_stop]
          )

In [ ]:
model_loss = pd.DataFrame(model.history.history)
model_loss.plot()

<img src='img\dropout.png'/>

***

## Testing the model
#### Using model.evaluate()

##### link : https://colab.research.google.com/drive/1m2cg3D1x3j5vrFc-Cu0gMvc48gWyCOuG#forceEdit=true&sandboxMode=true&scrollTo=nb4_EtfK5DuW

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=1) 

print('Test accuracy:', test_acc)

This will print out the accuracy of our model when using it with a test set,
overfitting can also be noticed here

***

## Making Predictions 
#### with testing_data

In [ ]:
predictions = model.predict(
      x=scaled_test_samples
    , batch_size=10
    , verbose=0
)  

unlike with training and validation sets, we do not pass the labels of the test set to the model during the inference stage.

To see what the model's predictions look like, we can iterate over them and print them out.

In [ ]:
for i in predictions:
    print(i)

In [ ]:

[ 0.74106830  0.25893170]
[ 0.14958295  0.85041702]
[ 0.96918124  0.03081879]
[ 0.12985019  0.87014979]
[ 0.88596725  0.11403273]
...

We can also look only at the most probable prediction

In [ ]:
rounded_predictions = np.argmax(predictions, axis=-1)

for i in rounded_predictions:
    print(i)

In [ ]:
0
1
0
1
0
...

however, we cannot judge how accurate these predictions are just by looking at the predicted output.
If we have corresponding labels for the test set , then we can compare these true labels to the predicted labels to judge the accuracy of the model's evaluations
using **Confusion Matrix**

- link on creating matrix : https://deeplizard.com/learn/video/km7pxKy4UHU

# Confusion Matrix for model evaluation

In [ ]:
predictions = model.predict_classes(X_test)

from sklearn.metrics import classification_report,confusion_matrix
# Classification report เหมือนเวลาดูใน Weka
print(classification_report(y_test,predictions))

# print simple confusion matrix
print(confusion_matrix(y_test,predictions))